In [ ]:
#The code shown in this section is the code used to scrape LinkedIn profiles. The code used to scrape
#the profiles in the database returns the scraped information from the profiles (name, description,
#location, experience, education and URL, without the data points containing dates and locations in the
#scraping of the experience and education sections). 

import csv
import requests
from selenium import webdriver
from time import sleep
from bs4 import BeautifulSoup
from lxml import etree
from parsel import Selector
import pandas as pd

# read the profiles, save the data
writer = csv.writer(open('database.csv', 'w'))
writer.writerow(['name', 'description', 'location', 'experience', 'education', 'ln_url'])

driver = webdriver.Chrome('/Users/Gonzalo Rodríguez/chromedriver')
driver.get('https://www.linkedin.com/login')
sleep(1)

# Put your LinkedIn email here
username_input = driver.find_element("name", 'session_key')
username_input.send_keys('grodriguez@inveready.com')

# Put your LinkedIn password here
password_input = driver.find_element("name", 'session_password')
password_input.send_keys('password')
sleep(2)

# Click on the sign in button
driver.find_element("xpath", '//button[text()="Iniciar sesión"]').click()
sleep(2)

# List LinkedIn profiles
lst = []
for profile in profile_urls:
    driver.get(profile)
    sleep(2)

    try:
        soup2 = BeautifulSoup(driver.page_source, 'html.parser')
        dom = etree.HTML(str(soup2))
        sel = Selector(text=driver.page_source)

        # Get name
        try:
            target_name = soup2.find("h1", {"class": "text-heading-xlarge inline t-24 v-align-middle break-words"})
            name = target_name.get_text().strip()
        except:
            name = ''

        # Get description
        try:
            target_title = soup2.find("div", {"class": "text-body-medium break-words"})
            title = target_title.get_text().strip()
        except:
            title = ''

        # Get location
        try:
            tree = etree.ElementTree(dom)
            find_text = etree.XPath("//div[@class='application-outlet']")
            for target in find_text(dom):
                loc = tree.getpath(target)
                location_xpath = loc + '/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[2]/div[2]/span[1]/text()'
                location = dom.xpath(location_xpath)[0].strip()
        except:
            location = ''
        location = location.strip()
        ln_url = profile

    # Get Experience
    # Find section
    tree = etree.ElementTree(dom)
    find_text = etree.XPath("//div[@id='experience']")
    for target in find_text(dom):
        pos_exp = tree.getpath(target)
        list1 = [i for i in pos_exp]
        list1[-2] = str(int(pos_exp[-2]) + 2)
        if 'navigation-index-see-all-experiences' in str(soup2):
            list1.append('/div/div/div/a')
            pos_exp = ''.join(list1)
            # Go to see all experiences, count experiences
            driver.find_element("xpath", pos_exp).click()
            sleep(2)
            soup2 = BeautifulSoup(driver.page_source, 'html.parser')
            dom = etree.HTML(str(soup2))
            sel = Selector(text=driver.page_source)
            sel_exp = str(soup2)
            num_exp = sel_exp.count('<li class="pvs-list__paged-list-item artdeco-list__item \
                                    pvs-list__item--line-separated pvs-list__item--one-column"')
            i = 0
            experiences = []
            while i < num_exp:
                # Identify if it is a position with promotions (reads differently)
                i = i + 1
                tree = etree.ElementTree(dom)
                find_text = etree.XPath("//div[@class='application-outlet']")
                for target in find_text(dom):
                    pos_exp = tree.getpath(target)
                    listi1 = pos_exp + '/div[3]/div/div/div[2]/div/div/main/section/div[2]/div/div[1]/ul' + '/li[' + str(i) + ']'
                    sel_li = ''
                    try:
                        sel_li = sel.xpath(listi1).extract()[0]
                    except:
                        sel_li = ''
                    if sel_li.count('li class="pvs-list__paged-list-item pvs-list__item--onecolumn"') > 1: # Has promotions
                        # Get info
                        listi1 = listi1 + '/div/div/div[2]'
                        company = listi1 + '/div[1]/a/div/div/div/div/span[1]/text()'
                        try:
                            company1 = dom.xpath(company)[0]
                        except:
                            company1 = ''
                        company1 = company1.strip()
                        position = listi1 + '/div[2]/ul/li/div/div/div[1]/ul/li[1]/div/div/div[2]/div[1]/a/div/div/div/div/span[1]/text()'
                        try:
                            position1 = dom.xpath(position)[0]
                        except:
                            position1 = ''
                        position1 = position1.strip()
                        
                        if sel_li.count('li class="pvs-list__paged-list-item pvs-list__item--onecolumn"') > 1:  # has promotions
                            # More code for promotions
                            # ...
                        else:  # has no promotions
                            listi1 = listi1 + '/div/div/div[2]/div/div[1]'
                            position = listi1 + '/div/div/div/div/span[1]/text()'
                            try:
                                position1 = dom.xpath(position)[0]
                            except:
                                position1 = ''
                            position1 = position1.strip()
                            company = listi1 + '/span[1]/span[1]/text()'
                            try:
                                company1 = dom.xpath(company)[0]
                            except:
                                company1 = ''
                            company1 = company1.strip()
                        experiences.extend([position1, company1])
                        
                        # Go back if you are inside all experiences
                        driver.execute_script("window.history.go(-1)")
                        sleep(2)
                        soup2 = BeautifulSoup(driver.page_source, 'html.parser')
                        dom = etree.HTML(str(soup2))
                        sel = Selector(text=driver.page_source)
                    
                    else:
                        list1.append('/ul')
                        pos_exp = ''.join(list1)
                        sel_exp = sel.xpath(pos_exp).extract()[0]
                        # Find jobs (count experiences, add remaining xpath 'manually')
                        num_exp = sel_exp.count('<li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column"')
                        i = 0
                        experiences = []
                        while i < num_exp:
                            i = i + 1
                        # Identify if it is a position with promotions (reads differently)
                            listi1 = list1.copy()
                            listi1.append('/li[' + str(i) + ']')
                            pos_li = ''.join(listi1)
                            try:
                                sel_li = sel.xpath(pos_li).extract()[0]
                            except:
                                sel_li = ''
                            
                            if 'pvs-entity--with-path' in sel_li:  # has promotions
                                listi1.append('/div/div[2]')
                                company = listi1.copy()
                                company.append('/div[1]/a/div/div/div/div/span[1]/text()')
                                company = ''.join(company)
                                try:
                                    company1 = dom.xpath(company)[0]
                                except:
                                    company1 = ''
                                company1 = company1.strip()
                            
                                position = listi1.copy()
                                position.append('/div[2]/ul/li[1]/div/div[2]/div/a/div/div/div/div/span[1]/text()')
                                position = ''.join(position)
                                try:
                                    position1 = dom.xpath(position)[0]
                                except:
                                    position1 = ''
                                position1 = position1.strip()
                            else:  # has no promotions
                                listi1.append('/div/div[2]/div/div[1]')
                                position = listi1.copy()
                                position.append('/div/div/div/div/span[1]/text()')
                                position = ''.join(position)
                                try:
                                    position1 = dom.xpath(position)[0]
                                except:
                                    position1 = ''
                                position1 = position1.strip()
                                company = listi1.copy()
                                company.append('/span[1]/span[1]/text()')
                                company = ''.join(company)
                                try:
                                    company1 = dom.xpath(company)[0]
                                except:
                                    company1 = ''
                                company1 = company1.strip()
                                experiences.extend([position1, company1])
                            
                        # Get education
                        tree = etree.ElementTree(dom)
                        find_text = etree.XPath("//div[@id='education']")
                        for target in find_text(dom):
                            pos_edu = tree.getpath(target)
                            list2 = [i for i in pos_edu]
                            list2[-2] = str(int(pos_edu[-2]) + 2)
                            if 'navigation-index-see-all-education' in str(soup2):
                                list2.append('/div/div/div/a')
                                pos_edu = ''.join(list2)
                                # Go to see all education
                                driver.find_element("xpath", pos_edu).click()
                                sleep(2)
                                soup2 = BeautifulSoup(driver.page_source, 'html.parser')
                                dom = etree.HTML(str(soup2))
                                sel = Selector(text=driver.page_source)
                                sel_edu = str(soup2)
                                num_edu = sel_edu.count('<li class="pvs-list__paged-list-item artdeco-list__item \
                                                       pvs-list__item--line-separated pvs-list__item--one-column"')
                                i = 0
                                education = []
                                while i < num_edu:
                                    i = i + 1
                                    tree = etree.ElementTree(dom)
                                    find_text = etree.XPath("//div[@class='application-outlet']")
                                    for target in find_text(dom):
                                        pos_exp = tree.getpath(target)
                                        listi2 = pos_exp + '/div[3]/div/div/div[2]/div/div/main/section/div[2]/div/div[1]/ul' + '/li[' + str(i) + ']/div/div/div[2]/div[1]/a'
                                        school = listi2 + '/div/div/div/div/span[1]/text()'
                                        try:
                                            school1 = dom.xpath(school)[0]
                                        except:
                                            school1 = ''
                                        school1 = school1.strip()
                                        degree = listi2 + '/span[1]/span[1]/text()'
                                        try:
                                            degree1 = dom.xpath(degree)[0]
                                        except:
                                            degree1 = ''
                                        degree1 = degree1.strip()
                                        education.extend([school1, degree1])
                            else:
                                list2.append('/ul')
                                pos_edu = ''.join(list2)
                                sel_edu = sel.xpath(pos_edu).extract()[0]
                                num_edu = sel_edu.count('<li class="artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column"')
                                i = 0
                                education = []
                                while i < num_edu:
                                    i = i + 1
                                    listi2 = list2.copy()
                                    listi2.append('/li[' + str(i) + ']/div/div[2]/div/a')
                                    school = listi2.copy()
                                    school.append('/div/div/div/div/span[1]/text()')
                                    school = ''.join(school)
                                    try:
                                        school1 = dom.xpath(school)[0]
                                    except:
                                        school1 = ''
                                    school1 = school1.strip()
                                    degree = listi2.copy()
                                    degree.append('/span[1]/span[1]/text()')
                                    degree = ''.join(degree)
                                    try:
                                        degree1 = dom.xpath(degree)[0]
                                    except:
                                        degree1 = ''
                                    degree1 = degree1.strip()
                                    education.extend([school1, degree1])
                        
                            writer.writerow([name, title, location, experiences, education, ln_url])
                            lst.append([name, title, location, experiences, education, ln_url])
                        
                        except Exception as e:
                            print("Error occurred:", e)
                            continue
                        
                print(lst)
                driver.quit()
                                                      

        
